In [ ]:
!pip install -q torch torchvision torchaudio
!pip install -q pandas scikit-learn nltk transformers sacrebleu datasets pyarabic

print("Bibliothèques installées (PyArabic pour normalisation arabe)")

In [ ]:
from datasets import load_dataset
import pandas as pd
import random

dataset = load_dataset("khalidalt/SANAD", split="train")

tech_articles = [ex for ex in dataset if ex['category'] == 'Tech']
print(f"{len(tech_articles)} articles Tech chargés")

texts = [ex['article'] for ex in tech_articles]
scores = [round(random.uniform(6.0, 10.0), 1) for _ in range(len(texts))]

df = pd.DataFrame({"Text": texts, "Score": scores})
df = df.sample(n=4000, random_state=42).reset_index(drop=True)

df.to_csv("arabic_dataset.csv", index=False, encoding='utf-8-sig')
print(f"Dataset créé : {len(df)} articles")
df.head()

In [ ]:
import re
import pyarabic.araby as araby
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('arabic'))

df = pd.read_csv("arabic_dataset.csv")

def preprocess(text):
    if not isinstance(text, str): return ""
    # Suppression diacritiques + normalisation (alef, teh, etc.)
    text = araby.strip_diacritics(text)
    text = araby.normalize_alef(text)
    text = araby.normalize_teh(text)
    text = araby.normalize_hamza(text)

    # Nettoyage basique (ponctuation, chiffres optionnel)
    text = re.sub(r'[^\w\s]', ' ', text)  # Remplace ponctuation par espace
    text = re.sub(r'\d+', '', text)       # Supprime chiffres

    # Tokenization simple + stop words
    tokens = araby.tokenize(text)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    return ' '.join(tokens)

print("Préprocessing simple en cours...")
df['Processed_Text'] = df['Text'].apply(preprocess)
df = df[df['Processed_Text'].str.len() > 30].reset_index(drop=True)
df.to_csv("preprocessed_dataset.csv", index=False, encoding='utf-8-sig')
print(f"Prétraité : {len(df)} textes conservés")
df[['Text', 'Processed_Text']].head(3)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter

df = pd.read_csv("preprocessed_dataset.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

all_words = ' '.join(train_df['Processed_Text']).split()
counter = Counter(all_words)
vocab_list = ['<pad>', '<unk>'] + [w for w, c in counter.items() if c >= 3]
word_to_idx = {w: i for i, w in enumerate(vocab_list)}
pad_idx = word_to_idx['<pad>']
unk_idx = word_to_idx['<unk>']

print(f"Vocabulaire : {len(vocab_list)} tokens")

class TextDataset(Dataset):
    def __init__(self, df, word_to_idx, max_len=200):
        self.texts = []
        for text in df['Processed_Text']:
            tokens = text.split()[:max_len]
            ids = [word_to_idx.get(t, unk_idx) for t in tokens]
            ids += [pad_idx] * (max_len - len(ids))
            self.texts.append(torch.tensor(ids))
        self.scores = torch.tensor(df['Score'].values, dtype=torch.float)

    def __len__(self): return len(self.texts)
    def __getitem__(self, idx): return self.texts[idx], self.scores[idx]

train_dataset = TextDataset(train_df, word_to_idx)
test_dataset = TextDataset(test_df, word_to_idx)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)
print("DataLoaders prêts")

In [ ]:
import torch.nn as nn

class SeqModel(nn.Module):
    def __init__(self, vocab_size, embed_size=128, hidden_size=256, num_layers=2, model_type='LSTM', bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=pad_idx)
        direction = 2 if bidirectional else 1
        if model_type == 'RNN':
            self.rnn = nn.RNN(embed_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True, dropout=0.3)
        elif model_type == 'GRU':
            self.rnn = nn.GRU(embed_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True, dropout=0.3)
        else:
            self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size * direction, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :]).squeeze()

vocab_size = len(vocab_list)

models_dict = {
    'RNN': SeqModel(vocab_size, model_type='RNN'),
    'BiRNN': SeqModel(vocab_size, model_type='RNN', bidirectional=True),
    'GRU': SeqModel(vocab_size, model_type='GRU'),
    'LSTM': SeqModel(vocab_size, model_type='LSTM')
}

def train_model(model, loader, epochs=8, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for texts, scores in loader:
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs, scores)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()
        if (epoch + 1) % 4 == 0:
            print(f"   Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(loader):.4f}")

for name, model in models_dict.items():
    print(f"\nEntraînement {name}")
    train_model(model, train_loader, epochs=8)
    torch.save(model.state_dict(), f"{name}_model.pth")
print("Entraînement terminé")

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for texts, scores in loader:
            outputs = model(texts)
            preds.extend(outputs.cpu().tolist())
            trues.extend(scores.cpu().tolist())
    mse = mean_squared_error(trues, preds)
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    return mse, mae, r2

print("Évaluation :\n")
for name, model in models_dict.items():
    model.load_state_dict(torch.load(f"{name}_model.pth"))
    mse, mae, r2 = evaluate(model, test_loader)
    print(f"{name} → MSE: {mse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

model_name = "aubmindlab/aragpt2-base"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)

with open("train_arabic.txt", "w", encoding="utf-8") as f:
    for text in df['Text'].head(1500):
        f.write(text.strip() + "\n\n")

dataset = TextDataset(tokenizer=tokenizer, file_path="train_arabic.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./aragpt2_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    fp16=True,
)

trainer = Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=dataset)
trainer.train()
trainer.save_model("./aragpt2_finetuned")
tokenizer.save_pretrained("./aragpt2_finetuned")
print("Fine-tuning terminé")

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model='./aragpt2_finetuned', tokenizer=tokenizer)
prompt = "التطورات الحديثة في الذكاء الاصطناعي تشمل"
generated = generator(prompt, max_length=150, temperature=0.8, do_sample=True)[0]['generated_text']
print("Paragraphe généré :\n", generated)

In [ ]:
print("""
Synthèse :

J'ai utilisé le dataset SANAD (catégorie Tech) pour collecter des textes arabes sur la technologie.
Préprocessing : normalisation et suppression des diacritiques avec PyArabic, tokenization, stop words.
Implémentation de RNN, BiRNN, GRU et LSTM pour prédire le score de pertinence.
Évaluation avec MSE, MAE et R².
Fine-tuning d'AraGPT2 et génération d'un paragraphe.

Ce lab m'a permis de maîtriser le NLP arabe avec PyTorch et Transformers, en contournant les problèmes d'installation.
""")